In [ ]:
#이 프로그램은 Ensemble 테스트 전용으로 annotation 처리는 제외한다.
#신규방식인 각 모델 결과를 모아서 최적의 영역을 선택 추출
#이전 방식(한 모델의 결과를 걸러 다음 모델로 넘겨 테스트)은 new_RetinaNet - TEST.ipynb 참조

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from itertools import groupby
from itertools import chain
from operator import eq
import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import tensorflow as tf
import keras_retinanet.utils.new_util as my


OS_ENV = 'LINUX' # 'LINUX' or 'WIN'
MODEL_ENV = 'single' # 'ensemble' or 'separate'
DATE = '20190116_abnormal_with_merge'
DATA_TO_TEST = 'abn_all' # 'abn1' or 'abn2' or 'abn_all' or 'normal' or 'normal_1'
#normal_1': #2018. 1월말 데이터중 Normal 1318 file

#SCORE_VALUE_LIST = [0.5,0.4]
SCORE_VALUE_LIST = [0.4]

In [ ]:
if OS_ENV == 'LINUX':
    result_save_base_path = '/home/huray/workspace/han_work/han_test/results/'
else:
    result_save_base_path = 'D:/Work/NCC/han_test/results/'
result_save_subpath = os.path.join(result_save_base_path, DATE + '_' + MODEL_ENV, DATA_TO_TEST)
result_save_path = result_save_subpath
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

## Load RetinaNet model

In [ ]:
model_list = []
#---------------------------------resnet50---------------------------------------
#1 : 1e-5
#model_list.append('snapshots/resnet50/resnet50_csv_90.h5')
#2 : from ep90, 1e-5, 4376
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_03.h5')
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_19.h5')
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_27.h5')
#model_list.append('snapshots/resnet50/from_e90/resnet50_csv_36.h5')
#3 : from ep90, 1e-6, 4617(이전 비정상 테스트 데이터 추가 학습)
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_37.h5')
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_43.h5')
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_49.h5')
##model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_65.h5')
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_77.h5')
#model_list.append('snapshots/resnet50/from_e90_4617/resnet50_csv_80.h5')


#---------------------------------resnet101---------------------------------------
#1 : 1e-5
#model_list.append('snapshots/resnet101/resnet101_csv_61.h5')
###model_list.append('snapshots/resnet101/resnet101_csv_59.h5')
#2 : from ep61, 1e-5, 4376
#model_list.append('snapshots/resnet101/from_e61/resnet101_csv_35.h5')
#model_list.append('snapshots/resnet101/from_e61/resnet101_csv_29.h5')
#model_list.append('snapshots/resnet101/from_e61/resnet101_csv_56.h5')
#3 : from ep59, 1e-7, 4617(이전 비정상 테스트 데이터 추가 학습)
#model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_77.h5')
#model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_82.h5')
model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_90.h5')
#model_list.append('snapshots/resnet101/from_e59_4617/resnet101_csv_91.h5')


#---------------------------------resnet152---------------------------------------
#1 : 1e-5
#model_list.append('snapshots/resnet152/resnet152_csv_74.h5')
#2 : from ep74, 1e-5, 4376
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_32.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_45.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_53.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_57.h5')
#model_list.append('snapshots/resnet152/from_e74/resnet152_csv_84.h5')
#3 : from ep74, 1e-7, 4617(이전 비정상 테스트 데이터 추가 학습)
#model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_53.h5')
#model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_55.h5')
#model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_57.h5')
##model_list.append('snapshots/resnet152/from_e74_4617/resnet152_csv_71.h5')


####################################### 이전모델 ###################################
#ensemble 아래 e44 + e29
#single e29
#model_list.append('snapshots/0326_more_aug/resnet101_csv_44.h5')   
#model_list.append('snapshots/0525_add_ncc_mass_data_strong_aug/resnet101_csv_29.h5')
#######

#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_25.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_29.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_47.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_48.h5')
#model_list.append('snapshots/0620_stronger_aug/resnet101_csv_49.h5')



## Initialize data generators

In [ ]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TO_TEST == 'abn1':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn1.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn1_win.csv'
elif DATA_TO_TEST == 'abn2':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn2.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn2_win.csv'
elif DATA_TO_TEST == 'normal':
    if OS_ENV == 'LINUX':    
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_normal.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_normal_win.csv'
elif DATA_TO_TEST == 'normal_1': #2018. 1월말 데이터중 Normal 1381 file
    if OS_ENV == 'LINUX':    
        data_csv_path = '/home/huray/data/NCC_trainset/img_retinanet/data_ncc_normal_180130.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC_trainset/img_retinanet/data_ncc_normal_180130.csv'                
elif DATA_TO_TEST == 'abn_all':
    if OS_ENV == 'LINUX':
        data_csv_path = '/home/huray/data/NCC/img_retinanet/data_abn_merged.csv'
        #data_csv_path = '/home/huray/data/NCC/img_retinanet/0914_new_NCC_test_only_abn.csv'
        #data_csv_path = '/home/huray/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_imsi.csv'
    else:
        data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/data_abn_merged_win.csv'        
        #data_csv_path = 'D:/Work/NCC/data/NCC/img_retinanet/0914_new_NCC_test_only_abn_for_win.csv'        
else:
    print('WRONG DATA CSV PATH!')
    raise
    
if OS_ENV == 'LINUX':
    label_csv_path = '/home/huray/data/new_trainset/class_2_malig_or_norm.csv'
else:
    label_csv_path = 'D:/Work/NCC/data/new_trainset/class_2_malig_or_norm.csv'    
    

# create a generator for testing data
val_generator = CSVGenerator(
    data_csv_path,
    label_csv_path,
    transform_generator=val_image_data_generator,
    batch_size=1
)

########## csv로 부터 테스트 파일목록을 read ######
test_file_df = pd.read_csv(data_csv_path, header=None)
num_of_test_files = len(test_file_df)
print('total test file : {}'.format(num_of_test_files))

########### 테스트파일 Sort ######
test_file_list = list(test_file_df[0]) #csv에서 읽어온 테스트 이미지파일 정보 목록
test_file_list.sort(key=lambda x: '-'.join(x.split('-')[:2])) #테스트 파일 목록을 groupby 하기 위해서 정렬

## define and initialize

In [ ]:
number_abnormal_detect_from_normal = 0 #normal 파일에서 abnormal로 검출된 경우의 수
img_index_list = list(range(0, num_of_test_files))
total_num_image_file = len(img_index_list)

ordinal=lambda x:["1st","2nd","3rd","4th","5th","6th"][x-1]

############# for 테스트 파일목록 Groupping ############
groupby_key_list = [] #그룹명(테스트 파일 단위 그룹 명) 목록
groupby_file_list = [] #그룹화된 테스트파일 목록
groupby_detected_number_for_files = [] #검출(detect)된 수에 대한 그룹화된 목록(그룹화된 테스트 목록 구조와 동일하게 매핑)
#list_not_detected_group_index = [] #검출되지 않은 그룹의 index 목록 

#M_max_scores = [] #abnormal 테스트 파일에서 detect된 anchor score 중 가장 높은 score
#N_max_scores = [] #normal 각 테스트 파일에서 detect된 anchor score 중 가장 높은 score

#테스트 파일 목록을 group by하여 list에 저장
#groupby key(그룹핑 기준(단위)는 파일명을 '-'로 split 한뒤 앞쪽 2번째 요소까지 추출함)
for key, group in groupby(test_file_list, key=lambda x: '-'.join(x.split('-')[:2])):
    #그룹 명(key) 목록
    groupby_key_list.append(key) 
    temp = list(group)
    
    #그룹화된 전체 테스트파일 목록 
    groupby_file_list.append(temp) 
    
    #그룹화 된 테스트 파일별 검출정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    groupby_detected_number_for_files.append([0]*len(temp))
    
    #그룹화 된 테스트 파일별 검출된 최대 score정보 목록(그룹화된 전체 테스트파일 목록과 동일한 구조로 구성),0 으로 초기화
    #M_max_scores.append([0]*len(temp))
    
    
    
############################# !!!!!!!!!!!!!!!!!!!!!!!!!!! ############################# 
###테스트 결과 출력
def get_detection_info():
    list_not_detected_group_index = []  # 검출되지 않은 그룹의 index 목록
    total_not_detected_file = 0 #검출되지 않은 전체 테스트 파일의 수    
    i_group_member = 0

    print('------------------------------------------------ Result info --------------------------------------------------------')
    #print('\n')    
    
    for idx_x, row in enumerate(groupby_detected_number_for_files):
        i_detections = 0
        for idx_y, elem in enumerate(row):
            i_detections += elem
            if elem == 0:
                total_not_detected_file += 1
                
        if i_detections == 0:
            list_not_detected_group_index.append(idx_x)
            i_group_member += len(row)
            print('({})set have no detection!.'.format(groupby_key_list[idx_x]))

    #print('\n')

    #1st print type
    #print('Total Number of abnormal case not detected :::::::::: {}'.format(total_not_detected_file))
    #print('Number of abnormal case set not detected::::::: {}({} files)'.format(len(list_not_detected_group_index), i_group_member))

    #2nd print type
    #print('{}/{} files not detected.'.format(total_not_detected_file,len(test_file_list)))
    #print('{}({} files)/{} sets not detected.'.format(len(list_not_detected_group_index), i_group_member, len(groupby_key_list)))

    #3rd print type
    print('{} files'.format(total_not_detected_file))
    print('{}({} files) sets  not detected.'.format(len(list_not_detected_group_index), i_group_member))
    #print('\n')
    if 'normal' in DATA_TO_TEST:
        print('{} abnormal detected.(from {} normal files).'.format(number_abnormal_detect_from_normal,len(test_file_list)))
    
    print('---------------------------------------------------------------------------------------------------------------------')

###정보 초기화(일부 list 정보에 대한 초기화)
def init_info():
    for idx_i, row in enumerate(groupby_file_list):
        for idx_j, col in enumerate(row):
            groupby_detected_number_for_files[idx_i][idx_j] = 0
            #M_max_scores[idx_i][idx_j] = 0
            
    number_abnormal_detect_from_normal = 0

## Run detection

In [ ]:
model = []
model_count = 0;

for model_path in model_list:
    model_count += 1

    print('{} model {} load..'.format(ordinal(model_count), model_path.split('/')[-1]))
            
    # Load RetinaNet Models
    model.append(keras.models.load_model(model_path, custom_objects=custom_objects))

for SCORE_VALUE in SCORE_VALUE_LIST:
    print('****************{}*******************'.format(MODEL_ENV))
    print('*      Detecting with score {}           *'.format(SCORE_VALUE))
    print('*******************************************')
    
    print('{} test files.'.format(total_num_image_file))
        
    print("start.")
    start = time.time()
    for index_count, index in enumerate(img_index_list):
        number_of_proper_detection = 0 #검출된 수
        detect_list=[]
        detect_score_list=[]
        
        # load image
        image, image_path = val_generator.load_image(index, get_image_path=True)
        #print('image :::::::::: {}'.format(image_path))
        image_name_split = image_path.split('/')[-1].split('-')

        # copy to draw on
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

        # preprocess image for network
        image = val_generator.preprocess_image(image)
        image, scale = val_generator.resize_image(image)

        for cur_model in model:
            # process detection
            _, _, detections = cur_model.predict_on_batch(np.expand_dims(image, axis=0))

            # compute predicted labels and scores
            predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
            scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

            # correct for image scale
            detections[0, :, :4] /= scale

            for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
                if score < SCORE_VALUE or label != 0:
                    continue

                number_of_proper_detection += 1
                detect_list.append(detections[0, idx, :4].astype(int))
                detect_score_list.append(score)

        #이미지에서 (조건에 부합되게)detect된 것이 있을 경우
        if number_of_proper_detection > 0:
            #ensemble 모델들을 통해 얻은 선택 영역들에 대해 중복영역 등을 고려하여 최종 선택
            
            #1.검출영역 선택시 병합없이 선택
            #selected_detection_list_idxs = my.select_deteced_list(detect_list, detect_score_list)
            
            #2.검출영역 선택시 일정 조건에 따라 영역을 병합하며, Score는 병합된 영역의 평균으로 대체된다.
            selected_detection_list_idxs = my.select_deteced_list_wih_merge(detect_list, detect_score_list)
        
            for idxx, b in enumerate(detect_list):
                if selected_detection_list_idxs[idxx] == 0: #검출영역 정리되어 선택된 것만....
                    #cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
                    cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 2)
                    caption = "{} {:.3f}".format(val_generator.label_to_name(label), detect_score_list[idxx])
                    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
                    cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)

            '''
            # 테스트 파일 그룹 목록에서 현재 파일의 위치를 구함    
            img_indics = my.find(groupby_file_list, image_path)
            if img_indics[0] >= 0:
                # 검출된 정보 중 Max 스코어 값을 파일 위치와 매칭하여 저장
                M_max_scores[img_indics[0]][img_indics[1]] = M_max_score 
            '''

            #이미지에서 (조건에 부합되게)detect된 것이 있을 경우
            #if number_of_proper_detection > 0:
            if 'normal' in image_path:
                number_abnormal_detect_from_normal += 1            

            ##################### 그룹화된 테스트 파일 정보와 동일한 구조의 detection 정보 목록에 반영 ##############
            # 테스트 파일 그룹 목록에서 현재 파일의 위치를 구함    
            img_indics = my.find(groupby_file_list, image_path)
            if len(img_indics) > 0:
                #groupby_detected_number_for_files[img_indics[0]][img_indics[1]] += number_of_proper_detection
                groupby_detected_number_for_files[img_indics[0]][img_indics[1]] = 1


        ############검출된 정보를 이미지에 표시, 저장################
        #cv2.imwrite(os.path.join(result_save_path, '-'.join(image_path.split('/')[-2:])), draw)
        
        del detect_list
        del detect_score_list

        #print('{}/{} completed.\r'.format(index+1,num_of_test_files), end='')
        print('{}/{} completed.\r'.format(index_count+1,total_num_image_file), end='')

    print("end({:.2f} minutes elapsed).".format((time.time() - start)/60))

    # release the memory
    #del model
    #keras.backend.clear_session()
        
    get_detection_info() #detection 결과 출력
   
    print('\n')
    
    #다음 Score로 테스트하기 위해 이전 검출 정보를 초기화 해줘야 함...
    init_info()
    

In [ ]:
print('Finished!')